In [33]:
# N, M, F = map(int, input().split())
# matrix = [list(map(int, input().split())) for _ in range(N)]
# wall = [[list(map(int, input().split())) for _ in range(M)] for _ in range(5)]
# f_list = [list(map(int, input().split())) for _ in range(F)]
N, M, F = 8, 3, 2
matrix = [[4, 0, 0, 0, 0, 0, 0, 0],
          [0, 1, 1, 1, 1, 1, 0, 0],
          [0, 1, 3, 3, 3, 1, 0, 1],
          [0, 1, 3, 3, 3, 1, 0, 1],
          [0, 1, 3, 3, 3, 0, 0, 0],
          [0, 1, 1, 1, 1, 1, 1, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 1, 1]]
wall = [[[1, 1, 1], [0, 0, 0], [0, 1, 1]], 
        [[1, 1, 1], [1, 0, 1], [1, 1, 1]], 
        [[0, 0, 1], [1, 0, 0], [1, 0, 1]],
        [[0, 0, 0], [1, 0, 0], [1, 1, 1]], 
        [[2, 0, 0], [0, 1, 0], [0, 0, 0]]]
f_list = [[0, 7, 1, 14], [6, 3, 3, 2]]

sk_3d, si_3d, sj_3d = find_3d_start()
ei, ej = find_2d_end()
ek_3d, ei_3d, ej_3d, si, sj = find_3d_end_2d_start()
print(sk_3d, si_3d, sj_3d, ei, ej)
print(ek_3d, ei_3d, ej_3d, si, sj)

dist = bfs_3d(sk_3d, si_3d, sj_3d, ek_3d, ei_3d, ej_3d)
print(dist)
di = [0, 0, 1, -1]
dj = [1, -1, 0, 0]

if dist != -1:
    v = [[401] * N for _ in range(N)]
    for wi, wj, wd, wv in f_list:
        v[wi][wj] = 1
        for mul in range(1, N+1):
            wi, wj = wi+di[wd], wj+dj[wd]
            if 0<=wi<N and 0<=wj<N and matrix[wi][wj]==0 and (wi,wj)!=(ei,ej):
                if v[wi][wj] > wv*mul:
                    v[wi][wj] = wv*mul
            else:
                break
    dist = bfs_2d(v, dist, si, sj, ei, ej)
    
print(dist)

4 0 0 0 0
0 2 0 4 5
9
28


In [ ]:
def myprint_3d(arr3):
    for arr in arr3:
        for lst in arr:
            print(*lst)
        print()
    print()

def myprint_2d(arr):
        for lst in arr:
            print(*lst)
        print()

In [31]:
from collections import deque
def bfs_2d(v, dist, si, sj, ei, ej):
    queue = deque()
    queue.append((si, sj))
    v[si][sj] = dist
    
    while queue:
        ci, cj = queue.popleft()        
        if (ci, cj) == (ei, ej):
            return v[ci][cj]        
        for di, dj in ((-1, 0), (1, 0), (0, -1), (0, 1)):
            ni, nj = ci + di, cj + dj            
            if 0 <= ni < N and 0 <= nj < N and matrix[ni][nj] == 0 and v[ci][cj] + 1 < v[ni][nj]:
                v[ni][nj] = v[ci][cj] + 1
                queue.append((ni, nj))
    return -1
                
    
def bfs_3d(sk_3d, si_3d, sj_3d, ek_3d, ei_3d, ej_3d):
    left_nxt = {0: 2, 2: 1, 1: 3, 3: 0}
    right_nxt = {0: 3, 3: 1, 1: 2, 2: 0}
    queue = deque()
    visited = [[[0] * M for _ in range(M)] for _ in range(5)]
    queue.append((sk_3d, si_3d, sj_3d))
    visited[sk_3d][si_3d][sj_3d] = 1
    while queue:
        ck, ci, cj = queue.popleft()
        
        if (ck, ci, cj) == (ek_3d, ei_3d, ej_3d):
            return visited[ck][ci][cj] # 거리 반환
        
        # 네 방향, 범위내/ 범위밖이면 다른 평면
        for di, dj in ((-1, 0), (1, 0), (0, -1), (0, 1)):
            ni, nj = ci + di, cj + dj
            
            # 범위밖
            if ni < 0:
                if ck == 0: nk, ni, nj = 4, (M-1) - cj, M-1
                elif ck == 1: nk, ni, nj = 4, cj, 0
                elif ck == 2: nk, ni, nj = 4, (M-1), cj
                elif ck == 3: nk, ni, nj = 4, 0, (M-1) - cj
                elif ck == 4: nk, ni, nj = 3, 0, (M-1) - cj
            elif ni >= M:
                if ck == 4: nk, ni, nj = 2, 0, cj
                else: continue
            elif nj < 0:
                if ck == 4: nk, ni, nj = 1, 0, ci
                else:
                    nk, ni, nj = left_nxt[ck], ci, M-1
            elif nj >= M:
                if ck == 4: nk, ni, nj = 0, 0, (M-1) - ci
                else:
                    nk, ni, nj = right_nxt[ck], ci, 0
            else:
                nk = ck
                
            if visited[nk][ni][nj] == 0 and wall[nk][ni][nj] == 0:
                visited[nk][ni][nj] = visited[ck][ci][cj] + 1
                queue.append((nk, ni, nj))
    return -1
    

def find_3d_base():
    for i in range(N):
        for j in range(N):
            if matrix[i][j] == 3:
                return i, j
            
def find_3d_end_2d_start():
    bi, bj = find_3d_base()
    for i in range(bi, bi+M):
        for j in range(bj, bj+M):
            if matrix[i][j] != 3: continue
            elif matrix[i][j+1] == 0:
                return 0, M-1, (M-1) - (i-bi), i, j+1
            elif matrix[i][j-1] == 0:
                return 1, M-1, (i-bi), i, j-1
            elif matrix[i+1][j] == 0:
                return 2, M-1, (j-bj), i+1, j
            elif matrix[i-1][j] == 0:
                return 3, M-1, (M-1) - (j-bj), i-1, j
    return -1
            

def find_3d_start():
    for i in range(M):
        for j in range(M):
            if wall[4][i][j] == 2:
                return 4, i, j

def find_2d_end():
    for i in range(N):
        for j in range(N):
            if matrix[i][j] == 4:
                matrix[i][j] = 0
                return i, j